In [1]:
import os, tqdm, glob, pickle,re
import pandas as pd

import soynlp
# from soynlp.word import WordExtractor
from soynlp.tokenizer import  MaxScoreTokenizer#LTokenizer,MaxLRScoreTokenizer,

import matplotlib as mpl
import matplotlib.font_manager as fm

import matplotlib.pyplot as plt
plt.style.use('ggplot')
from collections import Counter
import nltk

In [2]:
os.chdir('./data')

In [3]:
plt.rcParams["font.family"] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 24.
plt.rcParams['xtick.labelsize'] = 20.
plt.rcParams['ytick.labelsize'] = 20.
plt.rcParams['axes.labelsize'] = 20.

티볼리와 코나 텍스트

In [11]:
with open('티볼리_docs_190214.lst','rb') as fp:
    docs0 = pickle.load(fp)

In [10]:
with open('코나_docs_190214.lst','rb') as fp:
    docs1 = pickle.load(fp)

In [12]:
doc_data0 = {'doc_id':[],'site_nm':[],'board_cat':[],'post_dt':[],'title':[], 'content':[]}
for doc in docs0:
    doc_data0['doc_id'].append(doc['doc_id'])
    doc_data0['site_nm'].append(doc['site_nm'])
    doc_data0['board_cat'].append(doc['board_cat'])
    doc_data0['post_dt'].append(doc['post_dt'])
    doc_data0['title'].append(doc['title'])
    doc_data0['content'].append(doc['content'])

In [13]:
doc_data1 = {'doc_id':[],'site_nm':[],'board_cat':[],'post_dt':[],'title':[], 'content':[]}
for doc in docs1:
    doc_data1['doc_id'].append(doc['doc_id'])
    doc_data1['site_nm'].append(doc['site_nm'])
    doc_data1['board_cat'].append(doc['board_cat'])
    doc_data1['post_dt'].append(doc['post_dt'])
    doc_data1['title'].append(doc['title'])
    doc_data1['content'].append(doc['content'])

In [14]:
doc_df =pd.concat((pd.DataFrame(doc_data0),pd.DataFrame(doc_data1)),axis=0)

In [15]:
del docs0
del doc_data0

In [16]:
del docs1
del doc_data1

In [ ]:
#################################

In [17]:
to_drop = doc_df[doc_df.duplicated(subset='doc_id')].index
doc_df.drop(index=to_drop,inplace=True)
doc_df.reset_index(drop=True,inplace=True)
doc_df.describe(include='object')

,doc_id,site_nm,board_cat,post_dt,title,content
count,59746,59746,59746,59746,59746,59746
unique,59746,385,1,54642,55215,57291
top,593fd113c12eb242e004b298,■ 코나 공식 동호회 ▣코나 패밀리▣ 소형 SUV 현대 코나 KONA : 네이버 카페,consumer,2017-06-12T15:00:00Z,안녕하세요,==============================================...
freq,1,12968,59746,47,232,36


In [18]:
doc_df.site_nm.value_counts()[:10]

■ 코나 공식 동호회 ▣코나 패밀리▣ 소형 SUV 현대 코나 KONA : 네이버 카페    12968
★티볼리 클럽★ 에어,가솔린,디젤,4륜,시승기,결함,튜닝,동호회                11723
[클럽] 티볼리 ★쌍용자동차 공식 동호회★                             5865
보배드림                                                4582
티볼리 클럽★아머,에어,가솔린,디젤,4륜,시승기,결함,동호회                   2465
디시인사이드                                              2348
[전기차동호회]현대아이오닉일렉트릭,테슬라모델3,볼트EV,코나EV                 2307
[전기차동호회]코나일렉트릭,니로EV,쉐보레볼트EV,아이오닉EV                  2064
뽐뿌                                                  1660
★티볼리멤버스★티볼리에어,가격표,시승기,디젤,가솔린,동호회                    1120
Name: site_nm, dtype: int64

In [19]:
doc_df = doc_df[pd.Series(map(len,doc_df.content))<10000] # 지나치게 긴글 제외
doc_df.reset_index(drop=True,inplace=True)
doc_df.describe(include='object')

,doc_id,site_nm,board_cat,post_dt,title,content
count,59742,59742,59742,59742,59742,59742
unique,59742,385,1,54638,55211,57287
top,593fd113c12eb242e004b298,■ 코나 공식 동호회 ▣코나 패밀리▣ 소형 SUV 현대 코나 KONA : 네이버 카페,consumer,2017-06-12T15:00:00Z,안녕하세요,==============================================...
freq,1,12968,59742,47,232,36


In [20]:
doc_df.site_nm.value_counts()[:10]

■ 코나 공식 동호회 ▣코나 패밀리▣ 소형 SUV 현대 코나 KONA : 네이버 카페    12968
★티볼리 클럽★ 에어,가솔린,디젤,4륜,시승기,결함,튜닝,동호회                11723
[클럽] 티볼리 ★쌍용자동차 공식 동호회★                             5865
보배드림                                                4581
티볼리 클럽★아머,에어,가솔린,디젤,4륜,시승기,결함,동호회                   2465
디시인사이드                                              2348
[전기차동호회]현대아이오닉일렉트릭,테슬라모델3,볼트EV,코나EV                 2306
[전기차동호회]코나일렉트릭,니로EV,쉐보레볼트EV,아이오닉EV                  2064
뽐뿌                                                  1660
★티볼리멤버스★티볼리에어,가격표,시승기,디젤,가솔린,동호회                    1120
Name: site_nm, dtype: int64

In [21]:
docs = []
for i in range(len(doc_df)):
    docs.append(doc_df.loc[i,'title']+'. '+doc_df.loc[i,'content'])

In [22]:
# 메모리..
corpus = []
bmarks = [0,]
bmark = 0
prog = 0
for i,doc in enumerate(docs):
#     if prog%1000 == 0:
#         print(prog)
    prog += 1
    sents = nltk.sent_tokenize(doc)
    bmark += len(sents)
    bmarks.append(bmark)
    
    docs[i] = [sent for sent in sents if len(sent)>4]
    corpus += [sent for sent in sents if len(sent)>4]

In [23]:
del docs

미리 학습된 토크나이저 사용

In [44]:
with open('word_scores_suv.pickle','rb') as fp:
    word_scores = pickle.load(fp)

In [45]:
with open('cohesion_scores_suv.pickle','rb') as fp:
    cohesion_scores = pickle.load(fp)

In [46]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(word_scores.items(), key=lambda x:word_score(x[1]), reverse=True)[:10]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

..     (5746, 0.906, 5.399)
함께     (6962, 0.887, 5.264)
^^     (3661, 0.938, 5.191)
;;     (492, 0.934, 5.109)
으로     (717, 0.708, 5.385)
합니다.     (11336, 0.832, 5.212)
...     (3091, 0.698, 5.372)
됩니다.     (2476, 0.918, 5.059)
때문에     (6973, 0.674, 5.367)
입니다.     (5622, 0.741, 5.250)


In [47]:
mtokenizer = MaxScoreTokenizer(scores = cohesion_scores)

In [48]:
mtokenizer.tokenize('asdasdㅏ랗티볼리') # 토크나이징 잘된다

['as', 'da', 'sd', 'ㅏ랗', '티볼리']

토큰 생성

In [49]:
m_tokens = [mtokenizer.tokenize(sent) for sent in corpus]

In [50]:
tokens = m_tokens

In [51]:
for i,words in enumerate(tokens):
    tmp_words = list(map(lambda word:re.sub(r'[^가-힣a-zA-Z0-9]','',word),words))
    tokens[i] = [word for word in tmp_words if len(word)>1]

In [52]:
len(tokens)

268260

KOSAC 감성사전 이용해 문서의 감성점수 내보자

In [53]:
os.chdir('./../lexicon')

In [54]:
# with open('readme.txt',encoding='utf-8') as fp:
#     for line in fp.readlines():
#         print(line)

In [55]:
polar = pd.read_csv('polarity.csv',encoding='utf-8')
uni_mask = pd.Index(list(map(lambda lst:len(lst)<2, polar.ngram.str.split(';'))))
polar = polar[uni_mask]
intens_mask =polar['max.prop'] > 0.5 # POS, NEG > 0.5
value_mask = pd.Index(map(lambda val:(val in ('POS','NEG')) ,polar[intens_mask]['max.value']))
polar = polar[intens_mask][value_mask]
polar.reset_index(drop=True,inplace=True)
polar.head()

,ngram,freq,COMP,NEG,NEUT,None,POS,max.value,max.prop
0,가*/JKS,1,0.0,0.000000,0.000000,0.000000,1.000000,POS,1.000000
1,가*/VV,3,0.0,0.000000,0.000000,0.000000,1.000000,POS,1.000000
2,가/VV,11,0.0,0.545455,0.181818,0.090909,0.181818,NEG,0.545455
3,가/VX,9,0.0,0.333333,0.000000,0.111111,0.555556,POS,0.555556
4,가격/NNG,3,0.0,0.000000,0.000000,0.000000,1.000000,POS,1.000000


In [56]:
polar.ngram = polar.ngram.str.replace(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]','')
polar.sort_values(['ngram','freq'],ascending=[True,False],inplace=True)
dup_idx = polar[polar.duplicated(subset='ngram')].index
polar.drop(index=dup_idx,inplace=True)
polar.reset_index(drop=True,inplace=True)
len(polar)

2572

In [57]:
len(tokens)

268260

In [58]:
#document 단위로 구분된 토큰이 필요함
doc_tokens = []
tmp_doc = []
for i, sent in enumerate(tokens):
    if i%10000==0:
        print(i, end='\t')
    if i in bmarks[1:]: # 새 document 시작
        doc_tokens.append(tmp_doc)
        tmp_doc = []
    
    tmp_doc += sent
    

0	10000	20000	30000	40000	50000	60000	70000	80000	90000	100000	110000	120000	130000	140000	150000	160000	170000	180000	190000	200000	210000	220000	230000	240000	250000	260000	

In [59]:
doc_tokens[2]

['티볼리',
 '에어',
 '티볼리',
 '에어',
 '차주',
 '님들',
 '혹시',
 '손세차',
 '가면',
 '티볼리',
 '에어',
 '중형',
 '차값',
 '받나요']

In [60]:
# 단어별 감성점수 평균으로 문서별 감성점수 계산
polar_lst = polar.ngram.tolist()
doc_scores = []
for doc_token in  doc_tokens:
    doc_score = 0
    for token in doc_token:        
        if token in polar_lst:
            polar_idx = polar_lst.index(token)
            pos_neg = polar.loc[polar_idx,'max.value']
            token_score = polar.loc[polar_idx,pos_neg]
            if pos_neg=='NEG':
                token_score = -token_score
            
            doc_score+=token_score
    doc_scores.append(doc_score/len(token))

In [61]:
os.chdir('./../data')

In [64]:
# 티볼리+코나 문서의 긍부정 score
print('평균 긍부정score:', sum(doc_scores)/len(doc_scores))
print('\n긍부정score 기초통계')
print(pd.Series(doc_scores).describe())

평균 긍부정score: 0.9100255166873843

긍부정score 기초통계
count    56713.000000
mean         0.910026
std          2.411051
min        -21.276052
25%          0.000000
50%          0.365795
75%          1.250000
max        103.586536
dtype: float64


In [65]:
def token_score(token): # 출현문서 수, 평균 문서 스코어, 긍정분류 비율 리턴
    global doc_tokens, doc_scores
    this_scores = []
    polar = []
    for token_lst, score in zip(doc_tokens,doc_scores):
        if token in token_lst:
            this_scores.append(score)
            if score < 0.36: # 중간값 밑으로는 부정으로 태깅
                polar.append('neg')
            elif score > 0.36: # 중간값 위로는 긍정으로 태깅
                polar.append('pos')
            else:
                polar.append('neu')
    return len(this_scores), sum(this_scores)/len(this_scores), Counter(polar)['pos']/(Counter(polar)['pos'] + Counter(polar)['neg'])

In [66]:
token_score('쌍용')

(8282, 2.2900776247798227, 0.786524993962811)

In [67]:
token_score('현대')

(9261, 3.0128756239004875, 0.8656732534283554)

In [68]:
token_score('기아')

(1685, 3.8504713976817855, 0.8094955489614243)

In [69]:
token_score('티볼리')

(25347, 0.9963964919598699, 0.5077329756174545)

In [70]:
token_score('코나')

(25595, 1.2585090944378934, 0.5618381462232817)